In [10]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using FiniteDifferences
using TopologyOptimizationHelper

In [11]:
# Our favorite parameters for testing
# Run to recompute random variables
L = 20
ε = rand(480) .* 11 .+ 1
δε = randn(480) * 1e-6
ω = 2π
δω = randn() * 1e-6
dir = randn(length(ε))

A, x = Maxwell1d(L, ε, ω)
new_ε_A, _ = Maxwell1d(L, ε + δε, ω)
new_ω_A, _ = Maxwell1d(L, ε, ω + δω)

M = length(x)
b = zeros(M)
b[M÷2] = 1;

In [12]:
# Exact ∇_ε_LDOS versus finite difference
LDOS, ∇LDOS = ∇_ε_LDOS(A, b, ω)
new_LDOS, _ = ∇_ε_LDOS(new_ε_A, b, ω)

@show new_LDOS - LDOS
∇LDOS' * δε

new_LDOS - LDOS = 1.737817164067936e-11


1.7377361804715755e-11

In [13]:
# Exact ∇_ε_ω₀ versus finite difference
val, _ = Arnoldi_eig(A, ε, ω, b)
new_val, _ = Arnoldi_eig(new_ε_A, ε + δε, ω, b)
gradient = Eigengradient(A, ε, ω, b)
@show sqrt(new_val) - sqrt(val)
@show dot(δε, gradient)

# Exact ∇_ε_ω₀ versus Richardson extrapolation 
f = α -> Maxwell_omega(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * gradient

sqrt(new_val) - sqrt(val) = -1.3364577444718861e-7 + 6.355746252917089e-11im
dot(δε, gradient) = -1.3364576942715878e-7 + 6.355746955321727e-11im
extrapolate_fdm(central_fdm(2, 1), f, 0) = (-0.1183036117123022 - 0.0001722573584995295im, 5.116760673730784e-11)


-0.11830361171170992 - 0.00017225735850513092im

In [14]:
# Exact ∇_ω_LDOS versus finite difference
∇LDOS = ∇_ω_LDOS(A, b, ε, ω)
LDOS = Just_LDOS(L, ε, ω, b)
new_LDOS = Just_LDOS(L, ε, ω + δω, b; ω_pml=ω)

@show new_LDOS - LDOS
∇LDOS * real(δω)

new_LDOS - LDOS = 1.5569247812724137e-10


1-element Vector{Float64}:
 1.5563210705550908e-10

In [17]:
# Exact improved gradient versus finite difference
ω₀ = Maxwell_omega(L, ε, ω, b)
A₀, _ = Maxwell1d(L, ε, real(ω₀); ω_pml=ω)
LDOS, ∇LDOS = Improved_∇_ε_LDOS(A₀, ε, real(ω₀), b; ω_pml=ω)
new_LDOS =  Just_Improved_LDOS(L, ε + δε, ω, b)
@show new_LDOS - LDOS
@show ∇LDOS' * δε

# Exact improved gradient versus Richardson extrapolation
f = α -> Just_Improved_LDOS(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * ∇LDOS

new_LDOS - LDOS = -1.4021117637156438e-8
∇LDOS' * δε = -1.4021195874245141e-8
extrapolate_fdm(central_fdm(2, 1), f, 0) = (0.0003083412152716399, 1.2294624411428312e-12)


0.00030834121409575